Datenvisualisierung hat mich schon immer fasziniert. Leider hatte ich nie einen wirklichen Grund dafür es einzusetzen. Und doch wollte ich es mal ausprobieren. Ich habe mich sehr von [Jemus](http://blog.jemu.name/tvshows/2015/04/so-cougar-town-ended/) inspirieren lassen. Da passiert natürlich viel viel mehr und ausführlicher. Ich habe erstmal das gemacht was ich auch verstehe :). Und zwar die Episoden Ratings darzustellen.

Als erstes importiere ich ein paar Sachen. Erstmal `choice` um per Zufall Farben aus einer Palette auszuwählen. Dann `pandas` für die `DataFrames`. Um die Daten aus der Trakt-API zu bekommen benutze ich [trakt.py](https://github.com/fuzeman/trakt.py). Wieso alles selber bauen wenn es dieses schöne Modul gibt? Dann benutze ich [Bokeh](http://bokeh.pydata.org/en/latest/) um das ganze darzustellen.

In [1]:
from random import choice

import pandas as pd
from trakt.tv import TVShow
from bokeh.plotting import figure, ColumnDataSource, output_notebook, show
from bokeh.models import HoverTool
from bokeh.palettes import Spectral9

Wir starten die Bokeh Ausgabe in einem jupyter-Notebook.

In [2]:
output_notebook()

BokehJS successfully loaded.

Hier ist die Funktion die die Daten besorgt, verarbeitet und darstellt.

In [3]:
def ratings(show_name):
    
    # Es wird die Show definiert
    tv_show = TVShow(show_name)
    
    # Um die Staffeln ausseinander zu halten, soll jede Episode eine unterschiedliche 
    # Staffelfarbe haben. Bokeh hat ein paar Farbpaletten dabei. Ich benutze Spectral9.
    # Dazu brauchen wir noch eine liste um alle Episoden-Staffel-Farben zu speichern
    colormap = Spectral9
    colors = []
    
    # Wir brauchen eine Episoden Liste
    episodes = []
    
    # Damit die Episoden hintereinander auf der X-Achse dargestellt werden, brauchen sie
    # eine fortlaufende Nummer
    episode_number = 1
    
    # Dann arbeiten wir uns mal durch die Staffeln
    for season in tv_show.seasons:
        
        # Season 0 beinhaltet die Specials. Die werden ingnoriert
        if season.season == 0:
            continue
        
        # Eine Staffel-Farbe wird per Zufall ausgewaehlt. Wir schmeissen immer die letzte Farbe
        # aus einer Kopie der Farbenliste damit sie nicht nochmal gewaehlt wird
        if colors:
            new_colormap = colormap[:]
            new_colormap.remove(colors[-1])
            color = choice(new_colormap)
            
        else:
            color = choice(colormap)
        
        # Die Episoden 
        for episode in season.episodes:
            
            # Wenn es eine Ausstrahlungsdatum gibt, wird der Zeit-Teil weggeschnitten
            if episode.first_aired:
                aired = episode.first_aired.split('T')[0]
            else:
                aired = None
            
            # Nun werden die gesammelten Daten als Tupel der Episodenliste angehangen
            episodes.append((episode_number,
                             episode.season,
                             '{}.{}'.format(episode.season,
                                            episode.number), 
                              episode.title, 
                              aired, 
                              episode.rating))
            
            # Die Episoden-Staffel-Farbe wird der Farbenliste angehangen
            colors.append(color)
            
            # Die Episodennummer wird erhoeht
            episode_number += 1
                
    # Es wird ein DataFrame erstellt
    df = pd.DataFrame(
        episodes, 
        columns=['episode_number', 'season', 'episode', 'title', 'aired', 'rating'])

    # Es wird ein source-Objekt gebraucht damit die tooltips funktionieren.
    # Der DataFrame genuegt nicht um die Daten fuer die tooltips zu benutzen
    source = ColumnDataSource(data=df)
    
    # Wir erstellen in figure und belabeln die Achsen
    fig = figure(title=show_name)
    fig.xaxis.axis_label = 'Episodes'
    fig.yaxis.axis_label = 'Rating'
    
    # Hier bauen wir den Scatter-Plot. Wir uebergeben X und Y plus unser source-Objekt
    # und Style Geschichten. Unter anderem unsere Episoden-Farbenliste
    fig.scatter(df['episode_number'], df['rating'], 
                source=source, name='main', 
                fill_alpha=0.8, line_color='#000000', size=10, color=colors)
    
    # Ein HoverTool-Object in dem wir das Layout fuer die Tooltips festlegen
    hover = HoverTool(tooltips=[('Episode', '@episode'),
                                ('Title', '@title'),
                                ('Rating', '@rating'),
                                ('Aired', '@aired')])
    
    # Das Tool wird hinzugefuegt
    fig.add_tools(hover)
    
    # Und nun angezeigt
    show(fig)

Ich gebe eigentlich nicht viel auf die Ratings. Ich selber habe über Trakt noch nie etwas bewertet. Trotzdem handelt es sich um gute Daten um ein paar Sachen auszuprobieren.

Hier ein paar Ratings von Serien die mit entweder viel bedeuten oder die ich gerade schaue.

## Roswell

In [4]:
ratings('Roswell')

Die erste Serie bei der ich von einem persönlichen Fandom sprechen kann. Noch bevor ich wusste das es das Wort gibt. Ich habe damals sogar alle Bücher gelesen. Kurz hatte ich nachgedacht sie wegzuschmeissen. Leicht peinlich berührt meiner Serienliebe von damals. Habe mich doch dagegen entschieden. Die erste Staffel war grossartig. Der perfekte Mix aus Highschool und SciFi/Mystery. Danach nahm es für mich drastisch ab. Die dritte Staffel habe ich noch nicht einmal schauen können. Zu gross war die Angst vor dem Zerstören meiner Liebe zur Serie.

## Veronica Mars

In [5]:
ratings('Veronica Mars')

Vielleicht meine absolute Lieblingsserie. Veronica Mars habe ich vielleicht 4 oder 5 mal durchgeschaut. Und ich glaube dabei wird es nicht bleiben. Es so eine Serie die immer herhalten muss wenn es mir mal nicht so gut geht. Ein paar Folgen Veronica Mars sind immer eine gute Ablenkung. Alle drei Staffeln haben ungefähr die gleiche Bewertung. Dies deckt sich auch mit meiner Meinung. Drei gleich starke Staffeln. Wieso nur wurde die Serie abgesetzt? Das College-Szenario hätte noch soviel zu bieten gehabt. 

## Akte X

In [6]:
ratings('The X-Files')

Mit einem Knall wurde auf einmal eine zehnte Staffel angekündigt und gestern lief davon die dritte Folge. Auch wenn ich nur ein wenig mit Akte X vertraut bin, die drei Episoden haben mir bis jetzt sehr gefallen. Anscheinend alle Staffeln mit gleichen Höhen und Tiefen.

## Downton Abbey

In [7]:
ratings('Downton Abbey')

Ich hätte nicht gedacht das mich diese Geschichte so packt und emotional macht. Doch Downton Abbey wurde für mich zu einer Herzensangelegenheit. Jeden Staffeln mit viel Tränen. Den Abfall der Ratings nach hinten decken sich nicht mit meiner Meinung. Ich fand alle 6 ziemlich stark.

## Californication

In [8]:
ratings('Californication')

Eher so mal reingeschnuppert weil auf Netflix. Teilweise sehr unterhaltsam. Sonst kann ich nicht viel darüber sagen. Bin auch erst bei Staffel 4. Danach war ich wieder gesund und hatte keine Zeit mehr ;-).

## Modern Family

In [9]:
ratings('Modern Family')

Oft sage ich "Alles was ich über Familie weiss habe ich aus Modern Family". Viele Probleme kennt man selber und kann sie so doch mit sehr viel Humor von aussen betrachten. Mit das Beste was ich die letzten Jahre gesehen habe.

## Simpsons

In [10]:
ratings('The Simpsons')

Die erste Folge der ersten Deutschlandausstrahlung sogar als Kind gesehen. Bis dahin haben mich die Simpsons immer begleitet. Mal aktiver mal weniger aktiv. Mich wundert der geringe Ausschlag der Ratings. Nach hinten fand ich die Serie doch ziemlich schwach. Interessant ist, die schlechtesten Folgen sind die Clip-Folgen. Empfand ich auch immer so.

## Lilyhammer

In [11]:
ratings('Lilyhammer')

Gerad erst angefangen zu schauen. Gefällt mir sehr gut bis jetzt. Bin in der Mitte von Staffel Zwei.

## Sopranos

In [12]:
ratings('The Sopranos')

Denke ich an Lilyhammer muss ich natürlich auch an die Sopranos denken. Der gute Steven Van Zandt. Ich hätte nie gedacht das ich eine Serie über die Mafia so gut finden würde. Ich wurde eines besseren belehrt. Alles in mich aufgesogen. Hammer. Ist jetzt schon ein paar Jahre her und die Serie sitzt in DVD-Boxen in Schrank und wartet auf ein Wiedersehen.

## Community

In [13]:
ratings('Community')

Ich liebe Community und ich bin mir sicher das ich bis zur vierten Staffel kaum etwas witzigeres gesehen habe. Voll mit Popkultur und das in einem "College"-Setting. Einer der Serien die ich mir immer wieder anschauen kann, und doch nicht alles entdecke was da so drin steckt.

## Twin Peaks

In [15]:
ratings('Twin Peaks')

Und nochmal ein Klassiker. Nach dem der Mord um Laura Palmer geklärt war, ging es gefühlt richtig Berg ab. Ich bin auf jeden Fall auf die neue Staffel gespannt. Eine Fortsetzung die die Erwartungen kaum erfüllen kann.